<a href="https://colab.research.google.com/github/mario-ruoff/neural-network-learning/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Simple Convolutional Neural Network

#### Data Loader

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from IPython.display import display, clear_output

# Downloading MNIST dataset
print("Loading dataset...")
train_data = datasets.MNIST(
    root="dataset",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.MNIST(
    root="dataset",
    train=False,
    download=True,
    transform=ToTensor(),
)

# Hyperparameters
learning_rate = 0.01
batch_size = 1
# epochs = 5

# Set data loader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
print("Done")

Loading dataset...
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 39070840.58it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1081665.27it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9722725.61it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2973857.13it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Done


#### Network Setup

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class MarioNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cpu device


#### Network Training

In [3]:
# Model
model = MarioNet().to(device)
print(model)

# Training
model.train()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for batch, (X, y) in enumerate(train_dataloader):
    # Forward pass
    X, y = X.to(device), y.to(device)
    logits = model(X)
    loss = loss_function(logits, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Visualize
    if batch % 100 == 0:
        loss, current = loss.item(), batch * batch_size + len(X)
        clear_output(wait=True)
        display(f"loss: {loss:>7f}  [{current:>5d}/{len(train_dataloader.dataset):>5d}]")

'loss: 0.057687  [ 5601/60000]'

KeyboardInterrupt: 

#### Test Network

In [33]:
test_loss = 0
accuracy = 0
predictions = []
labels = []
with torch.no_grad():
    for X, y in test_dataloader:
        logits = model(X)
        labels.append(y)
        predictions.append(logits)
        test_loss += loss_function(logits, y).item()
        if logits.argmax(1) == y: accuracy += 1

test_loss /= len(test_dataloader)
accuracy /= len(test_dataloader.dataset)
print(f"Test loss: {test_loss}")
print(f"Accuracy: {accuracy * 100}%")

Test loss: 0.11822569660611007
Accuracy: 96.58%


#### Analysis

In [1]:
index = 1
# pred_list = pred_probab.squeeze().tolist()
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred.squeeze()}")
print("Label: ", labels[index], "- Prediction:", predictions[index].argmax(1))
plt.imshow(test_data.data[index].squeeze(), cmap=plt.cm.gray)
plt.show()
prediction = predictions[index].squeeze().tolist()
plt.bar(range(len(prediction)), prediction)

NameError: name 'labels' is not defined